In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

import acquire
import prepare
import features

from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score

from sklearn.feature_selection import SelectKBest, f_regression, RFE

from sklearn.linear_model import LinearRegression, LassoLars

from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = acquire.acquire_cache_data()

KeyError: 'None of [0] are in the columns'

In [ ]:
df.head()